In [1]:
# imports
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])

# Add to sys.path
sys.path.insert(0, parent_dir)

In [2]:
from utils.experiment_utils import get_all_experiments_info, load_best_model
import torch
import os
import hydra
from omegaconf import DictConfig, OmegaConf

import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from notebooks.mnist_classifier.mnist_tiny_cnn import TinyCNN

from mixer.mixer import SetMixer
from datasets.mnist import MNISTDataset

from torch.utils.data import DataLoader
from itertools import product

from tqdm.notebook import tqdm

from datasets.distribution_datasets import GaussianMixtureModelDataset
from utils.gmm_utils import fit_gmm_batch

from ot.gmm import gmm_ot_loss

import warnings
warnings.filterwarnings("ignore")

In [12]:
device = 'cuda'
configs = get_all_experiments_info('/orcd/data/omarabu/001/njwfish/DistributionEmbeddings/outputs/', False)
cfg = [
    c for c in configs if 'gmm_sys' in c['name']
    and c['config']['training']['num_epochs'] == 200
]

# load model and move to device
def load_model(cfg, path, device):
    enc = hydra.utils.instantiate(cfg['encoder'])
    gen = hydra.utils.instantiate(cfg['generator'])
    state = load_best_model(path)
    enc.load_state_dict(state['encoder_state_dict'])
    gen.model.load_state_dict(state['generator_state_dict'])
    enc.eval()
    gen.eval()
    enc.to(device)
    gen.to(device)
    return enc, gen

In [13]:
cfg

[{'name': 'gmm_systematic_exp_fb3f1bafd8959cfb09ed0b676e2c143a',
  'dir': '/orcd/data/omarabu/001/njwfish/DistributionEmbeddings/outputs/gmm_systematic_exp_fb3f1bafd8959cfb09ed0b676e2c143a',
  'config': {'dataset': {'_target_': 'datasets.distribution_datasets.MultivariateNormalDistributionDataset', 'n_sets': 50000, 'set_size': '${experiment.set_size}', 'data_shape': [2], 'seed': '${seed}', 'prior_mu': [0, 5], 'prior_cov_df': 10, 'prior_cov_scale': 1}, 'encoder': {'_target_': 'encoder.encoders.DistributionEncoderTx', 'in_dim': '${dataset.data_shape[0]}', 'latent_dim': '${experiment.latent_dim}', 'hidden_dim': '${experiment.hidden_dim}', 'set_size': '${experiment.set_size}', 'layers': 2, 'heads': 4}, 'model': {'_target_': 'layers.MLP', 'in_dims': [2, 32, 1], 'hidden_dim': 128, 'out_dim': 2, 'layers': 4}, 'generator': {'_target_': 'generator.ddpm.DDPM', 'model': '${model}', 'betas': [0.0001, 0.02], 'n_T': 400, 'drop_prob': 0.1, 'noise_shape': '${dataset.data_shape}'}, 'optimizer': {'_targ

In [14]:
d = {
    "Encoder" : [],
    "Generator" : [],
    "N dims" : [],
    "OT reconstruction error" : []
}

N_sets = 40
set_size = 10**3


for c in tqdm(cfg):
    encoder_name = c['encoder']
    generator_name = c['generator']
    data_shape = c['config']['dataset']['data_shape']
    num_epochs = c['config']['training']['num_epochs']

    if num_epochs != 200:
        continue

    if 'KME' in encoder_name or 'Mean' in encoder_name:
        continue

    try:
        enc, gen = load_model(c['config'], c['dir'], device=device)
    except:
        print(encoder_name)
        continue

    if data_shape[0] != 2:
        continue

    

    if 'Tx' not in encoder_name and 'Wormhole' not in encoder_name:
        set_size = 10**5
    else:
        set_size=10**3

    dataset = GaussianMixtureModelDataset(
        n_sets=N_sets,
        set_size=set_size,
        prior_mu=(0,5),
        data_shape=data_shape
    )
    
    ot_errors = []
    for idx in range(len(dataset)//8):

        with torch.no_grad():
            x = torch.tensor(dataset.data[idx*8:(idx+1)*8], dtype=torch.float).cuda()
            z = enc(x)
            x_hat = gen.sample(z, num_samples=10**5)

        mus = dataset.mu[idx*8:(idx+1)*8]
        covs = dataset.cov[idx*8:(idx+1)*8]
        weights = dataset.weights[idx*8:(idx+1)*8]


        r_means, r_covs, r_weights = fit_gmm_batch(
            x_hat.detach().cpu().numpy(), 
            0.5 * mus,
            covs,
            weights,
            # use_kmeans_init=True
        )

        ot_dists = [
            gmm_ot_loss(r_m, m, r_c, c, r_w, w) 
            for r_m, m, r_c, c, r_w, w in zip(r_means, mus, r_covs, covs, r_weights, weights)
        ]
        ot_errors += ot_dists

    d['Encoder'].append(encoder_name)
    d['Generator'].append(generator_name)
    d['N dims'].append(data_shape[0])
    d['OT reconstruction error'].append(np.mean(ot_errors))
    print(f"Encoder: {encoder_name}, Generator: {generator_name}, OT error: {np.mean(ot_errors)}, data shape: {data_shape[0]}")

  0%|          | 0/6 [00:00<?, ?it/s]

Encoder: DistributionEncoderTx, Generator: DDPM, OT error: 17.922968574795117, data shape: 2
Encoder: DistributionEncoderGNN, Generator: CVAE, OT error: 18.602895371552073, data shape: 2
Encoder: WormholeEncoder, Generator: WormholeGenerator, OT error: 17.918656918880316, data shape: 2
Encoder: DistributionEncoderGNN, Generator: DirectGenerator, OT error: 14.822601921486296, data shape: 2
Encoder: DistributionEncoderTx, Generator: CVAE, OT error: 16.623401321714727, data shape: 2
Encoder: DistributionEncoderGNN, Generator: DDPM, OT error: 16.34861585789228, data shape: 2


In [4]:
pd.DataFrame(d).sort_values(by='OT reconstruction error')

,Encoder,Generator,N dims,OT reconstruction error
1,DistributionEncoderTx,DirectGenerator,10,8.776316e+02
0,DistributionEncoderTx,CVAE,10,1.126318e+03
3,DistributionEncoderTx,DirectGenerator,10,6.721938e+03
4,WormholeEncoder,WormholeGenerator,10,8.771564e+03
5,DistributionEncoderGNN,CVAE,10,3.744157e+04
2,DistributionEncoderTx,DDPM,10,7.753786e+04
6,DistributionEncoderGNN,DirectGenerator,10,3.222186e+06
7,DistributionEncoderGNN,DirectGenerator,10,8.801756e+07
